# Модуль 1: Введение в Observability и Distributed Tracing
---

## Вступление

Здравствуйте, студенты! Сегодня мы начинаем наш курс по наблюдаемости в распределённых системах, и первое, о чём мы поговорим, — это **Observability** и **Distributed Tracing**. Эти технологии стали стандартом де-факто в современной разработке микросервисов, и вот почему:

> Вопрос: *Если пользователь говорит, что «приложение тормозит», как понять, **где именно** тормозит?*

* Проблема в БД?
* Слишком медленный запрос к стороннему API?
* Или просто один из микросервисов работает неэффективно?

Раньше мы могли отследить это с помощью логов. Сейчас — микросервисов десятки, сотни, и логи не дают полной картины. Мы не видим **весь путь запроса**. Поэтому нужен **трейсинг**.

---











## Часть 1. Что такое Observability?

### Определение:

**Observability** (наблюдаемость) — это способность системы ответить на вопрос:

> *«Что происходит внутри неё только по внешним данным?»*

### 3 «столпа» наблюдаемости:

1. **Логи (Logs)** — текстовые записи событий.
2. **Метрики (Metrics)** — числовые показатели (время ответа, нагрузка на CPU и т.п.).
3. **Трейсы (Traces)** — путь одного запроса через всю систему.




---

## Часть 2. Что такое Distributed Tracing?

Когда один пользовательский запрос проходит через 5-10 микросервисов, мы теряем понимание, **где** именно возникла задержка. Distributed tracing помогает восстановить цепочку вызовов:

> Представьте: пользователь отправил запрос, он прошёл через frontend → API Gateway → service A → service B → service C → и вернулся.

С помощью трейсинга мы можем:

* **визуализировать путь** запроса;
* **измерить** время на каждом этапе;
* **найти узкие места** (например, медленный вызов в B);
* **связать** трейсы с логами и метриками.


---

## Часть 3. Основные понятия трейсинга

* **Trace** — вся цепочка вызовов (один пользовательский запрос).
* **Span** — отдельная операция внутри trace (например, `getUserData` в сервисе).
* **Trace ID** — уникальный идентификатор всей цепочки.
* **Span ID** — уникальный ID каждой операции.
* **Parent ID** — ссылка на родительский span.

Пример:

```
Trace ID: abc123
└── Span 1: Запрос в API Gateway
    └── Span 2: Вызов service A
        └── Span 3: Вызов service B
```

Каждый `span` содержит:

* название операции (например, `GET /users`)
* длительность
* атрибуты (например, `http.status_code`, `db.statement`)
* ошибки, если они были

---

## Часть 4. Инструменты трейсинга: обзор

### OpenTelemetry

* Открытый стандарт для сбора данных наблюдаемости (трейсы, метрики, логи).
* Поддерживает множество языков: Python, Java, Go, Node.js и др.
* Может отправлять данные в Jaeger, Zipkin, Prometheus, Datadog и т.д.

### Jaeger

* Система визуализации трейсов.
* Позволяет видеть: время выполнения операций, зависимости между сервисами, ошибки.
* Поддерживается CNCF (как и Kubernetes!).

---

## Демонстрация

Открываем Jaeger UI, загружаем трейс. Смотрим:

* Какие сервисы участвуют
* Где задержка
* Где ошибка

🔧 Пример с e-commerce системой: пользователь оформил заказ → мы видим цепочку вызовов через frontend, payment, inventory и delivery.

---

---
## Закрепление материала

### Обсуждение:

* Почему логи недостаточны в микросервисной архитектуре?
* Что даст нам Trace ID в логах?
* В каких случаях трейсинг может не помочь?
---

---
## Практическое задание №1

1. **Теория**: Найдите и прочитайте статью о трейсинге в продакшене от одной из крупных компаний (Uber, Netflix, Spotify и др.).
2. **Визуализация**: Посмотрите видео-демо Jaeger или OpenTelemetry Collector.
3. **Вопрос на засыпку**: Как связать логи, метрики и трейсы вместе?
---


---
# Модуль 2: OpenTelemetry — архитектура и основы

## Вступление

Добро пожаловать на второе занятие!

На прошлом уроке мы разобрались, **зачем** нужен трейсинг. Теперь давайте посмотрим, **как** это реализуется в реальных системах. Сегодня мы познакомимся с OpenTelemetry — современным стандартом сбора наблюдаемости.




---

## Часть 1. Что такое OpenTelemetry?

**OpenTelemetry (OTel)** — это **open-source фреймворк**, предоставляющий инструменты и API для сбора **трейсов**, **метрик** и **логов** из приложений. Он разрабатывается под эгидой CNCF (как и Kubernetes) и становится промышленным стандартом.

> Цель OpenTelemetry — быть единым интерфейсом для сбора данных наблюдаемости и их экспорта в любые системы.



---

## Часть 2. Архитектура OpenTelemetry

OpenTelemetry состоит из **трёх основных уровней**:

### 1. **Instrumentation** (Инструментирование кода)

* Это уровень, где в ваш код вставляются **hooks** (автоматически или вручную).
* Сюда относятся SDK и библиотеки для языков (Python, Java, Go, Node.js...).
* Основная задача — **создавать трейсы (traces)**, метрики и логи.

 *Пример*: при входе в Flask-роут автоматически создаётся `Span`.

---

### 2. **Collector** (Сборщик данных)

* Это отдельный процесс, который принимает данные от ваших приложений.
* Может **агрегировать**, **фильтровать**, **обогащать**, **преобразовывать** и **отправлять** их дальше.
* Полностью конфигурируемый YAML-файлом.

 Это как центральный коммутатор, который управляет, куда пойдут данные.

---

### 3. **Backends** (Приёмники/визуализаторы)

* Это системы, в которые вы отправляете данные: Jaeger, Prometheus, Datadog, Zipkin, OTLP, Tempo, New Relic и др.
* OpenTelemetry не зависит от них — это плюс!

---



## Типичный поток данных

```
[Ваш код с SDK] → [OTel Collector] → [Jaeger / Prometheus / ...]
```

 Пример на практике:

* Python-сервис собирает трейсы с помощью `opentelemetry-sdk`
* Отправляет их по OTLP протоколу в Collector
* Collector перенаправляет их в Jaeger для визуализации

---

---
## Часть 3. Виды данных в OpenTelemetry

### Traces

* Цепочки вызовов между сервисами
* Формируются из Spans

### Metrics

* Количественные показатели: задержка, загрузка, кол-во запросов
* Часто идут в Prometheus или Cloud Monitoring

### Logs

* Обычные логи, но с привязкой к TraceID (это ключ к Observability 3-в-1)




---

## Часть 4. Collector: установка и конфигурация

### Установка (Docker):

In [ ]:
# docker-compose.yml
otel-collector:
  image: otel/opentelemetry-collector:latest
  command: ["--config=/etc/otel-collector-config.yml"]
  volumes:
    - ./otel-collector-config.yml:/etc/otel-collector-config.yml
  ports:
    - "4317:4317"  # gRPC OTLP
    - "55681:55681"  # HTTP OTLP

---
### Пример конфигурации (`otel-collector-config.yml`):

In [ ]:
receivers:
  otlp:
    protocols:
      grpc:
      http:

exporters:
  jaeger:
    endpoint: "http://localhost:14250"
    tls:
      insecure: true

service:
  pipelines:
    traces:
      receivers: [otlp]
      exporters: [jaeger]



 Здесь Collector принимает OTLP (от приложений), и пересылает трейсы в Jaeger.

---



## Часть 5. Режимы работы Collector

OpenTelemetry Collector можно использовать в 3 сценариях:

| Сценарий              | Описание                                                             |
| --------------------- | -------------------------------------------------------------------- |
| **Agent**             | Устанавливается рядом с приложением (например, в pod в Kubernetes).  |
| **Gateway**           | Централизованный сбор данных (в k8s через DaemonSet или Deployment). |
| **Sidecar + Gateway** | Комбинированный подход: локальный Agent + центральный Gateway.       |




---

## Демонстрация

* Покажем простой `Flask`-сервис, который создает трейсы через OpenTelemetry SDK.
* Подключим Collector и отправим трейсы в Jaeger.
* Откроем Jaeger UI и увидим запросы и спаны.



---

## Практическое задание

1. Установите OpenTelemetry Collector через Docker или бинарник.
2. Настройте его для приёма OTLP и отправки в Jaeger (можно использовать Docker Compose).
3. Напишите простой сервис (на любом языке, лучше Python или Node.js), который делает HTTP-запрос.
4. Добавьте OpenTelemetry SDK, чтобы отправлять трейсы в Collector.



---

## Вопросы для самопроверки:

1. Какие компоненты входят в архитектуру OpenTelemetry?
2. В чём различие между SDK и Collector?
3. Какие преимущества использования Collector в режиме Gateway?
4. Почему важно использовать OTLP, а не "нативные" протоколы?


---

## В завершение

Сегодня вы:

* Познакомились с архитектурой OpenTelemetry
* Научились запускать Collector и настраивать экспорт трейсов
* Сделали первый шаг к полной Observability в своих микросервисах

---

# Модуль 3: Jaeger — система распределённого трейсинга

---

## Вступление

Приветствую вас на третьем занятии!

Мы уже знаем, **зачем** нужен трейсинг, и как **OpenTelemetry** может собирать данные. Но где нам их **смотреть**? Где удобно анализировать и искать проблемы?

Сегодня мы познакомимся с **Jaeger** — мощным инструментом для визуализации и анализа распределённых трейсов.

---

## Часть 1. Что такое Jaeger?

**Jaeger** — это open-source система, разработанная Uber, для:

* трассировки запросов в микросервисной архитектуре,
* измерения задержек на разных этапах запроса,
* поиска узких мест и ошибок в цепочке вызовов.

Jaeger входит в экосистему CNCF и отлично интегрируется с OpenTelemetry.



---

## Часть 2. Архитектура Jaeger

Jaeger состоит из нескольких компонентов:

### 1. **Agent**

* Устанавливается рядом с приложением (на том же узле).
* Получает спаны через UDP и передаёт их коллектору.

### 2. **Collector**

* Принимает трейсы от агентов или напрямую от приложений (например, через OpenTelemetry).
* Отправляет данные в хранилище (базу данных).

### 3. **Storage** (Хранилище)

* По умолчанию используется **Elasticsearch**, **Cassandra** или **Badger**.
* Можно также использовать **Jaeger с встроенным in-memory хранилищем** (для тестов/демо).

### 4. **Query**

* HTTP-сервис, который получает запросы из браузера и ищет трейсы в хранилище.

### 5. **UI**

* Веб-интерфейс, где вы видите цепочки вызовов и детали каждого `span`.

---

## 🗺️ Визуальная схема

```
          +-----------+
          |   App     |
          +-----------+
               |
               |  OTLP / UDP
               v
          +-----------+
          | Collector |
          +-----------+
               |
               v
          +-----------+
          | Storage   |
          +-----------+
               ^
               |
          +-----------+     +--------+
          |  Query    |<--->|  UI    |
          +-----------+     +--------+
```

---



## Часть 3. Установка Jaeger (через Docker Compose)

Вот минимальный `docker-compose.yml`:

In [ ]:
version: '3'
services:
  jaeger:
    image: jaegertracing/all-in-one:latest
    ports:
      - "16686:16686"  # Web UI
      - "14250:14250"  # OTLP gRPC
      - "4317:4317"    # OTLP (OpenTelemetry)
    environment:
      COLLECTOR_OTLP_ENABLED: "true"


 После запуска UI доступен по адресу: `http://localhost:16686`


---

## Часть 4. Как работать с Jaeger UI

Открываем интерфейс:

* Вводим название сервиса (`my-flask-app`, `payment-service` и т.д.)
* Выбираем временной диапазон
* Видим список трейсов (с сортировкой по длительности, ошибкам)
* Кликаем на нужный трейс — открывается **визуализация дерева вызовов**

Что вы увидите:

* Общее время выполнения запроса
* Все спаны внутри
* Вложенность вызовов (parent → child)
* Тайминги каждого спана
* Ошибки и лог-сообщения




---

## Часть 5. Практика: первый трейсовый маршрут

1. Запускаем Jaeger из Docker Compose
2. Пишем простое Python-приложение с OpenTelemetry:

```bash
pip install opentelemetry-sdk opentelemetry-exporter-otlp opentelemetry-instrumentation-flask
```

3. Код:


In [ ]:
from flask import Flask
from opentelemetry.instrumentation.flask import FlaskInstrumentor
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.exporter.otlp.proto.grpc.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.trace.export import BatchSpanProcessor
from opentelemetry.sdk.resources import SERVICE_NAME, Resource

app = Flask(__name__)
FlaskInstrumentor().instrument_app(app)

provider = TracerProvider(resource=Resource.create({SERVICE_NAME: "demo-service"}))
provider.add_span_processor(BatchSpanProcessor(OTLPSpanExporter(endpoint="http://localhost:4317", insecure=True)))
from opentelemetry import trace
trace.set_tracer_provider(provider)

@app.route("/")
def hello():
    return "Hello, traced world!"

if __name__ == "__main__":
    app.run()


4. Переходим по `http://localhost:5000/`, затем смотрим трейсы в `http://localhost:16686`




---

## Часть 6. Анализируем и находим узкие места

### Что искать в трейсе:

* Спаны, которые занимают слишком много времени
* Ошибочные операции (`status.code != 0`)
* Неожиданные последовательности вызовов
* Места, где можно параллелить выполнение

---


## Частые вопросы

| Вопрос                                | Ответ                                                |
| ------------------------------------- | ---------------------------------------------------- |
| Что если трейсы не приходят в Jaeger? | Проверьте endpoint и OTLP протокол                   |
| Можно ли хранить трейсы дольше?       | Да, с внешним хранилищем (Cassandra, Elasticsearch)  |
| Как связать трейсы с логами?          | Используйте TraceID / SpanID в логах (log injection) |

---


---

## Практическое задание

1. Установите Jaeger с помощью Docker Compose.
2. Запустите свой Python/Node.js/Go сервис с поддержкой OpenTelemetry.
3. Отправьте один запрос, и найдите его в Jaeger.
4. Нарисуйте схему трейсинга (спан → вложенные спаны) по результату в UI.



---

## Итоги занятия

Сегодня вы:

* Поняли, как работает Jaeger
* Научились разворачивать Jaeger локально
* Подключили сервис к OTLP и увидели реальные трейсы
* Научились пользоваться UI-интерфейсом

---



# 🎓 Модуль 4: Интеграция OpenTelemetry с приложениями

---

## Вступление

Добро пожаловать на четвёртое занятие!

Теперь, когда мы знаем, **что такое трейсинг**, **как работает OpenTelemetry**, и как **настроить Jaeger**, самое время научиться подключать реальные приложения к этому процессу. Сегодня мы сделаем именно это!

Мы научимся:

* Встраивать **автоматическое** и **ручное** трассирование в Python, Node.js, Java
* Создавать и связывать **spans** вручную
* Добавлять **атрибуты**, **ошибки**, **контекст запроса**
* Передавать трейсинг между микросервисами



---

## Часть 1. Два способа интеграции

### 1. **Автоматическое инструментирование**

OpenTelemetry сам "вставляет" трейсинг в популярные библиотеки:

* HTTP-серверы (Flask, Express, FastAPI, Spring)
* Базы данных (PostgreSQL, MongoDB, Redis)
* gRPC, Kafka, HTTP-клиенты и др.

Плюсы:

* быстро
* не нужно лезть в код

Минусы:

* не охватывает бизнес-логику
* нет кастомных данных

---

### 2. **Ручное (явное) инструментирование**

Вы сами создаёте `spans` в нужных местах:

```python
with tracer.start_as_current_span("process_payment"):
    # ваша логика
```

Плюсы:

* можно трассировать внутренние блоки
* можно добавлять контекст

Минусы:

* нужно писать код



---

## Часть 2. Практика: Python + Flask

### Установка зависимостей:

```bash
pip install flask opentelemetry-sdk \
    opentelemetry-exporter-otlp \
    opentelemetry-instrumentation-flask
```

---

### Автоматическое трассирование Flask:


In [ ]:
from flask import Flask
from opentelemetry.instrumentation.flask import FlaskInstrumentor
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.exporter.otlp.proto.grpc.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.trace.export import BatchSpanProcessor
from opentelemetry.sdk.resources import SERVICE_NAME, Resource

# Настройка трейсера
trace.set_tracer_provider(TracerProvider(
    resource=Resource.create({SERVICE_NAME: "flask-app"})
))
tracer = trace.get_tracer_provider().get_tracer(__name__)
span_processor = BatchSpanProcessor(OTLPSpanExporter(endpoint="http://localhost:4317", insecure=True))
trace.get_tracer_provider().add_span_processor(span_processor)

# Flask-приложение
app = Flask(__name__)
FlaskInstrumentor().instrument_app(app)

@app.route("/")
def index():
    return "Hello, traced world!"

app.run(port=5000)


---

## Добавляем ручной span:


In [ ]:
@app.route("/checkout")
def checkout():
    with tracer.start_as_current_span("checkout-logic"):
        # Пример логики
        time.sleep(0.5)
        return "Checked out"


В Jaeger появится `checkout-logic` внутри HTTP-запроса `/checkout`


---

## Часть 3. Атрибуты, ошибки, контекст

### Атрибуты:

In [ ]:
span = trace.get_current_span()
span.set_attribute("user.id", 123)
span.set_attribute("cart.items", 5)


### Ошибки:


In [ ]:
try:
    1 / 0
except Exception as e:
    span.record_exception(e)
    span.set_status(Status(StatusCode.ERROR, str(e)))


---

## Часть 4. Передача контекста между сервисами

Когда один сервис вызывает другой, важно передать **Trace ID**, чтобы цепочка осталась целой.

OpenTelemetry делает это автоматически:

* Через HTTP заголовки (например, `traceparent`)
* С помощью middleware или клиентских библиотек

---

## 💡 Пример: два микросервиса на Flask

**Сервис A (`localhost:5000`) вызывает сервис B (`localhost:6000`)**

### A:



In [ ]:
import requests
@app.route("/call-b")
def call_b():
    with tracer.start_as_current_span("call-B-span"):
        response = requests.get("http://localhost:6000/hello")
        return response.text

### B:

In [ ]:
@app.route("/hello")
def hello():
    return "Hello from B"


✅ Благодаря автоматическому `instrumentation` `requests`, Trace ID будет передан.



---

## Часть 5. Другие языки

### Java (Spring Boot)

```xml
<!-- pom.xml -->
<dependency>
  <groupId>io.opentelemetry</groupId>
  <artifactId>opentelemetry-exporter-otlp</artifactId>
</dependency>
```

Spring позволяет добавить OTel через автоконфигурацию и agent:

```bash
java -javaagent:opentelemetry-javaagent.jar \
  -Dotel.exporter.otlp.endpoint=http://localhost:4317 \
  -Dotel.service.name=my-spring-app \
  -jar myapp.jar
```

---

### Node.js (Express)

```bash
npm install @opentelemetry/sdk-node \
  @opentelemetry/auto-instrumentations-node \
  @opentelemetry/exporter-trace-otlp-grpc
```

```js
const { NodeSDK } = require('@opentelemetry/sdk-node');
const { OTLPTraceExporter } = require('@opentelemetry/exporter-trace-otlp-grpc');
const sdk = new NodeSDK({
  traceExporter: new OTLPTraceExporter(),
  serviceName: 'node-app',
});
sdk.start();
```



---

## Часть 6. Лучшие практики

* ✅ Всегда добавляйте `SERVICE_NAME` в ресурс (resource attributes)
* ✅ Не пересоздавайте `TracerProvider` в каждом модуле
* ✅ Оборачивайте критические бизнес-функции вручную
* ✅ Передавайте контекст между сервисами
* ❌ Не записывайте чувствительные данные в трейсы



---

## Практическое задание

1. Напишите микросервис на любом языке (Python, Node.js, Java).
2. Настройте автоматическое трассирование HTTP-запросов.
3. Добавьте хотя бы один кастомный `span` внутри функции.
4. Добавьте к нему 2-3 `attribute` (например, user\_id, product\_id).
5. Сымитируйте ошибку и зафиксируйте её в трейсе.
6. Найдите этот трейс в Jaeger и проанализируйте его.



---

## 📌 Итоги

Сегодня вы научились:

* Интегрировать OpenTelemetry в приложения
* Создавать трейсы вручную и автоматически
* Передавать трейсинг между микросервисами
* Обогащать трейсы метаданными и ошибками

---


---

# Модуль 5: Анализ трейсинга и поиск узких мест в микросервисах

---

## Вступление

Добро пожаловать на занятие по **анализу трейсов**!

Теперь, когда ваши сервисы собирают трейсинг и данные уже появляются в Jaeger, встаёт вопрос:

> *Как эффективно их использовать, чтобы улучшать систему и отлаживать ошибки?*

Сегодня мы ответим на это. Вы научитесь **читать трейсы**, **выявлять аномалии**, и использовать Jaeger как **производственную лупу**.

---




## Часть 1. Вспомним: что такое трейс

Трейс = вся цепочка вызова от клиента до базы, через все микросервисы.

Пример:
👤 Пользователь нажал «Оформить заказ» →
API Gateway → Auth → Cart → Payments → Notification → БД

Каждое звено этой цепочки = **Span**
Все вместе = **Trace**

---



## Часть 2. Открываем Jaeger UI и читаем трейсы

Переходим по адресу `http://localhost:16686`

### Что мы видим:

* **Service** — имя микросервиса
* **Operation** — путь или метод (например, `GET /checkout`)
* **Duration** — общее время запроса
* **Trace Timeline** — визуализация дерева вызовов

---

### Пример структуры трейса:

```
Trace ID: abc123 (450ms)
├── API Gateway (10ms)
│   └── Auth Service (40ms)
├── Cart Service (100ms)
│   └── DB Query (70ms)
├── Payment Service (250ms)
│   └── External API Call (240ms)
```

---



## Часть 3. Поиск задержек

1. **Сортировка трейсов по Duration**
   → показывает медленные вызовы

2. **Фильтр по тегам**
   Например: `http.status_code=500`, `error=true`

3. **Просмотр критичных спанов**

   * Выделяются на таймлайне
   * Самые длинные спаны — ваш кандидат на оптимизацию

---

### 🔎 Что искать:

| Проблема                     | Как выглядит в трейсе                       |
| ---------------------------- | ------------------------------------------- |
| Медленная БД                 | Спан `SELECT ...` >100ms                    |
| Медленный сторонний API      | Внешний HTTP-запрос занимает много времени  |
| Потеря связи между сервисами | Разрывы цепочек (trace не продолжается)     |
| Перегрузка                   | Много трейсов с одинаковой точкой зависания |

---



## Часть 4. Поиск ошибок

1. Ошибки помечаются флагом `error = true`

2. Также можно фильтровать по:

   * `status.code != OK`
   * наличию исключений
   * кастомным атрибутам (`order.failed = true`)

3. Внутри спана можно открыть вкладку **Logs**:

   * Там будут события вроде `exception`, `db.timeout`, `retry`

---



## Часть 5. Кейсы из реального мира

### Кейс 1: медленная оплата

* `/checkout` занимает 1.3 секунды
* В трейсе видно, что `payment` → `ExternalPaymentAPI` = 1.2 сек
* 🛠 Решение: асинхронный вызов + кеширование валют

---

### Кейс 2: нестабильная авторизация

* 10% трейсов `/login` = `error=true`
* Спан `AuthService → Redis` возвращает `timeout`
* 🛠 Решение: добавлена реплика Redis + мониторинг

---

### Кейс 3: пропущенные Trace ID

* Некоторые трейсы не отображаются полностью
* У `orders` нет родительского спана
* 🛠 Причина: frontend не передаёт TraceContext

---



## Часть 6. Советы по анализу

* Всегда **начинайте с самого длинного трейса**
* Обратите внимание на **вложенность спанов** (глубокая — потенциальная проблема)
* Проверяйте наличие **ошибок в логах внутри спана**
* Используйте **теги и атрибуты** для фильтрации
* Используйте **Trace ID** как "ссылку" между трейсами, логами и алертами

---



## Практическое задание

1. Отправьте 5–10 запросов к вашему микросервису (например, `/checkout`, `/login`, `/order`)
2. Найдите трейсы в Jaeger
3. Ответьте:

   * Какой самый длинный спан?
   * Есть ли ошибки?
   * Где было больше всего времени потрачено?
4. Нарисуйте схему запроса на основе трейса (можно в draw\.io или от руки)

---

## Итоги занятия

Сегодня вы научились:

* Читать трейсы в Jaeger
* Выявлять узкие места и ошибки
* Использовать трейсы для отладки и оптимизации



---

# 🎓 Модуль 6: Интеграция OpenTelemetry в Kubernetes и CI/CD пайплайны

---

## Вступление

Итак, вы уже умеете:

✅ Добавлять трейсинг в код
✅ Отправлять его в Jaeger
✅ Анализировать ошибки и задержки

Теперь представьте:

> А что если система будет **собирать эти данные сама**, во всех окружениях — включая staging и production?

Сегодня вы узнаете, **как сделать observability частью вашей DevOps-инфраструктуры**.

---



## Часть 1. OpenTelemetry Collector в Kubernetes

OpenTelemetry Collector — это компонент, который:

* 🔄 Принимает данные от приложений (traces, metrics, logs)
* 🔁 Преобразует и обогащает
* 📤 Отправляет в backends (Jaeger, Prometheus, Loki и др.)

---

### Установка Collector в кластер (Helm)

```bash
helm repo add open-telemetry https://open-telemetry.github.io/opentelemetry-helm-charts
helm install my-otel-collector open-telemetry/opentelemetry-collector \
  --values my-values.yaml
```

**Пример конфигурации (`my-values.yaml`):**

In [ ]:
config:
  receivers:
    otlp:
      protocols:
        grpc:
        http:
  exporters:
    jaeger:
      endpoint: "jaeger-collector:14250"
      tls:
        insecure: true
  service:
    pipelines:
      traces:
        receivers: [otlp]
        exporters: [jaeger]


Теперь все данные, отправленные на `otel-collector:4317`, попадут в Jaeger.



---

## 📦 Часть 2. Автоматизация трейсинга в Kubernetes

### Подход 1: Автоинъекция через `sidecar` (например, для Istio или Linkerd)

* Istio автоматически добавляет прокси в под
* Прокси собирает трейсинг с каждого запроса
* Не нужно менять код

---

### Подход 2: Sidecar с OpenTelemetry Agent


In [ ]:
- name: otel-agent
  image: otel/opentelemetry-collector-contrib
  args: ["--config=/etc/otel/config.yaml"]
  volumeMounts:
    - mountPath: /etc/otel
      name: otel-config


Подходит для микросервисов, которые не поддерживают SDK

---



## Часть 3. Интеграция с CI/CD пайплайном

### Цель:

* Добавить трейсинг на каждый этап CI/CD
* Видеть, как деплой влияет на производительность
* Автоматически находить регрессии

---

### Jenkins / GitLab / GitHub Actions

В каждый job добавляем curl/post-запрос с trace context:

```bash
export TRACE_ID=$(uuidgen)
curl -X POST http://otel-collector:4318/v1/traces \
  -H "traceparent: 00-$TRACE_ID-0000000000000000-01" \
  -d @trace_payload.json
```

 Можно автоматически метить:

* `build_stage=start`
* `deployment_version=1.2.3`
* `ci.job=tests`

---



## Часть 4. Визуализация и наблюдение

OpenTelemetry позволяет выводить данные в:

* 🔍 **Jaeger** — трейсинг
* 📈 **Prometheus + Grafana** — метрики
* 📜 **Loki + Grafana** — логи
* ☁️ **Datadog, New Relic, AWS X-Ray** — коммерческие решения

---

### Пример Dashboard в Grafana:

* Количество запросов в минуту
* Средняя длительность трейсов
* Количество ошибок 500
* Топ-5 самых медленных endpoints

---



## Часть 5. Best Practices

| Что делать                                  | Почему                                  |
| ------------------------------------------- | --------------------------------------- |
| Используйте Collector как центральную точку | Позволяет гибко маршрутизировать данные |
| Отправляйте данные через OTLP               | Это рекомендуемый формат                |
| Используйте семантические теги              | Упростит анализ и фильтрацию            |
| Не пересылайте лишние трейсы                | Используйте sampling и фильтры          |
| Не храните трейсы вечно                     | Настройте TTL или архив                 |

---



## Практическое задание

1. Установите OpenTelemetry Collector в ваш кластер (minikube / kind / dev-cluster)
2. Настройте экспорт трейсов в Jaeger
3. Обновите один сервис с добавлением SDK (или используйте sidecar)
4. Отправьте запрос и проверьте trace в Jaeger
5. Попробуйте добавить трейс в CI (если есть доступ к GitLab/GitHub)

---

## Итоги курса

 Теперь вы умеете:

✅ Понимать, что такое Observability и зачем он нужен
✅ Использовать OpenTelemetry SDK и Collector
✅ Интегрировать с Jaeger и визуализировать трейсы
✅ Анализировать задержки и ошибки
✅ Встраивать наблюдаемость в продакшен и CI/CD

---


 Ниже — подробный мини-проект: мы создадим два микросервиса на Python (FastAPI), которые будут взаимодействовать друг с другом, собирать трассировки с помощью OpenTelemetry SDK и передавать их в Jaeger через OpenTelemetry Collector, работающий в Docker Compose.

---

# 🛠️ Мини-проект: "Tracing Microservices with OpenTelemetry + Jaeger"

## 📦 Состав проекта

```
otel-microservices/
│
├── service-a/                 # Первый микросервис (клиент)
│   └── main.py
│   └── requirements.txt
│
├── service-b/                 # Второй микросервис (обрабатывает запрос)
│   └── main.py
│   └── requirements.txt
│
├── otel-collector-config.yaml # Конфиг OpenTelemetry Collector
├── docker-compose.yaml        # Запуск всех компонентов
└── README.md                  # Инструкция по запуску
```

---


## 🐍 `service-a/main.py` (отправитель запроса)

In [ ]:
from fastapi import FastAPI
import requests
from opentelemetry import trace
from opentelemetry.instrumentation.requests import RequestsInstrumentor
from opentelemetry.instrumentation.fastapi import FastAPIInstrumentor
from opentelemetry.sdk.resources import Resource
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.trace.export import BatchSpanProcessor

# Tracing setup
resource = Resource(attributes={"service.name": "service-a"})
trace.set_tracer_provider(TracerProvider(resource=resource))
tracer = trace.get_tracer(__name__)
otlp_exporter = OTLPSpanExporter(endpoint="http://otel-collector:4318/v1/traces")
trace.get_tracer_provider().add_span_processor(BatchSpanProcessor(otlp_exporter))

app = FastAPI()
FastAPIInstrumentor().instrument_app(app)
RequestsInstrumentor().instrument()

@app.get("/")
def call_service_b():
    response = requests.get("http://service-b:8001/process")
    return {"msg": "Called Service B", "data": response.json()}

## 🐍 `service-b/main.py` (обрабатывает запрос)

In [ ]:
from fastapi import FastAPI
import time
from opentelemetry import trace
from opentelemetry.instrumentation.fastapi import FastAPIInstrumentor
from opentelemetry.sdk.resources import Resource
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.trace.export import BatchSpanProcessor

# Tracing setup
resource = Resource(attributes={"service.name": "service-b"})
trace.set_tracer_provider(TracerProvider(resource=resource))
otlp_exporter = OTLPSpanExporter(endpoint="http://otel-collector:4318/v1/traces")
trace.get_tracer_provider().add_span_processor(BatchSpanProcessor(otlp_exporter))
tracer = trace.get_tracer(__name__)

app = FastAPI()
FastAPIInstrumentor().instrument_app(app)

@app.get("/process")
def process():
    with tracer.start_as_current_span("fake_processing"):
        time.sleep(1.5)
    return {"result": "Processed by Service B"}

## 📄 `otel-collector-config.yaml`

In [ ]:
receivers:
  otlp:
    protocols:
      http:

exporters:
  jaeger:
    endpoint: "http://jaeger:14250"
    tls:
      insecure: true

service:
  pipelines:
    traces:
      receivers: [otlp]
      exporters: [jaeger]

## 🐳 `docker-compose.yaml`

version: "3.9"
services:

  service-a:
    build: ./service-a
    ports:
      - "8000:8000"
    depends_on: [otel-collector]
    command: uvicorn main:app --host 0.0.0.0 --port 8000

  service-b:
    build: ./service-b
    ports:
      - "8001:8001"
    depends_on: [otel-collector]
    command: uvicorn main:app --host 0.0.0.0 --port 8001

  otel-collector:
    image: otel/opentelemetry-collector-contrib:latest
    volumes:
      - ./otel-collector-config.yaml:/etc/otel/config.yaml
    command: ["--config=/etc/otel/config.yaml"]
    ports:
      - "4318:4318"

  jaeger:
    image: jaegertracing/all-in-one:latest
    ports:
      - "16686:16686"  # Web UI
      - "14250:14250"  # Collector gRPC

## 📦 `requirements.txt` (оба сервиса)

fastapi
uvicorn
requests
opentelemetry-api
opentelemetry-sdk
opentelemetry-exporter-otlp
opentelemetry-instrumentation
opentelemetry-instrumentation-fastapi
opentelemetry-instrumentation-requests


---

## Запуск проекта

```bash
git clone https://github.com/yourname/otel-microservices.git
cd otel-microservices
docker-compose up --build
```

Открой [http://localhost:8000](http://localhost:8000) — произойдет вызов Service B.
Перейди в [http://localhost:16686](http://localhost:16686) и выбери `service-a` — ты увидишь трейсинг двух сервисов.

---

## Что вы закрепите

* Использование OpenTelemetry SDK в FastAPI
* Визуализация трейсов в Jaeger
* Работа с Collector через OTLP
* Docker Compose + микросервисы

---